In [2]:
import pandas as pd
import numpy as np

import _pickle as pkl

# train_df=pd.read_csv('./complaints_train_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

from gensim.parsing.preprocessing import STOPWORDS
def remove_stopwords(tokens):
    # input and outputs a list of words
    return [word for word in tokens if word not in STOPWORDS]

train_df=pd.read_csv('./complaints_train_validation_data_clean.csv',usecols=["index","category_name","complaint_title","complaint_description",],na_filter=False)

whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')

sentences = (train_df['complaint_title']+" "+train_df['complaint_description']).tolist()
sentences = [''.join(filter(whitelist.__contains__, x)).lower() for x in sentences]
sentences = [x.split() for x in sentences]
sentences = list(map(remove_stopwords, sentences))
train_df.insert(1,"complaint_text",sentences)


val_df=pd.read_csv('./complaints_validation_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

test_df=pd.read_csv('./complaints_test_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

## gold labels
gold_df=pd.read_csv('./clean-gold-labels.tsv',sep='\t',usecols=["category_name","complaint_description",],na_filter=False)
gold_df['complaint_title'] = gold_df['complaint_description']
sentences = (gold_df['complaint_description']).tolist()
sentences = [''.join(filter(whitelist.__contains__, x)).lower() for x in sentences]
sentences = [x.split() for x in sentences]
sentences = list(map(remove_stopwords, sentences))
gold_df.insert(1,"complaint_text",sentences)
######

sentences = (test_df['complaint_title']+" "+test_df['complaint_description']).tolist()
sentences = [''.join(filter(whitelist.__contains__, x)).lower() for x in sentences]
sentences = [x.split() for x in sentences]
sentences = list(map(remove_stopwords, sentences))
test_df.insert(1,"complaint_text",sentences)

gold_df.head()

,category_name,complaint_text,complaint_description,complaint_title
0,Garbage,"[garbage, collector, coming, collect, househol...",Garbage collector is not coming to collect hou...,Garbage collector is not coming to collect hou...
1,New Bus Shelters,"[provide, bus, shelters, minimal, request, com...",Please provide bus shelters. Its the most mini...,Please provide bus shelters. Its the most mini...
2,Electricity,"[days, complete, power, cut, annasandra, palya...",2 days complete power cut in Annasandra Palya ...,2 days complete power cut in Annasandra Palya ...
3,Bad Roads,"[roads, dug, lay, drainage, pipes, laying, pip...",The roads were dug to lay the drainage pipes a...,The roads were dug to lay the drainage pipes a...
4,Land and Property Encroachment - Others,"[place, sites, connects, roads, sites, illegal...",Next to my place there are 2 sites which conne...,Next to my place there are 2 sites which conne...


In [15]:
colsize = len(train_df['category_name'])
train_df['category_name'] = train_df["category_name"].astype('category')
#train_df['true_label'] = pd.Series(np.zeros(colsize), index=train_df.index)
# train_df['predicted_label'] = pd.Series(np.zeros(colsize), index=train_df.index)

train_df['true_label'] = train_df['category_name'].cat.codes
# for i in range(colsize):
#     if(train_df['true_label'][i]==5):
#         print(train_df['true_label'][i],train_df['category_name'][i])

In [16]:
for i,x in enumerate(train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero}).index._data):
    print(i,x)

0 Air Pollution
1 Autorickshaws and Taxis
2 BMTC - Driver or Conductor
3 BMTC - Need new Bus Route
4 BMTC - Others
5 Bad Roads
6 Broken Storm Water Drains
7 Cattle
8 Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover
9 Desilting - Lakes
10 Diseases
11 Electricity
12 Flooding of Roads and Footpaths
13 Footpaths
14 Garbage
15 Government Land Encroachment
16 Hawkers and Vendors
17 Hoardings
18 Illegal posters and Hoardings
19 Lakes - Others
20 Maintenance of Roads and Footpaths - Others
21 Manholes
22 Mosquitos
23 Need New Streetlights
24 Need New Toilets 
25 New Bus Shelters
26 No Sewage Drains
27 Noise Pollution
28 Others
29 Overflow of Storm Water Drains
30 Parking Violations
31 Parks and playgrounds
32 Potholes
33 Public Nuisance
34 Repair of streetlights
35 Sewage and Storm Water Drains - Others
36 Stray Dogs
37 Traffic
38 Trees, Parks and Playgrounds - Others
39 Unauthorized Construction
40 Water Leakage
41 Water Supply 


In [17]:
class_names = train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero}).index.tolist()

pkl.dump(class_names,open("class_names.p","wb"))

noOfClasses = len(train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero}).index)

print(class_names)


['Air Pollution', 'Autorickshaws and Taxis', 'BMTC - Driver or Conductor', 'BMTC - Need new Bus Route', 'BMTC - Others', 'Bad Roads', 'Broken Storm Water Drains', 'Cattle', 'Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover', 'Desilting - Lakes', 'Diseases', 'Electricity', 'Flooding of Roads and Footpaths', 'Footpaths', 'Garbage', 'Government Land Encroachment', 'Hawkers and Vendors', 'Hoardings', 'Illegal posters and Hoardings', 'Lakes - Others', 'Maintenance of Roads and Footpaths - Others', 'Manholes', 'Mosquitos', 'Need New Streetlights', 'Need New Toilets ', 'New Bus Shelters', 'No Sewage Drains', 'Noise Pollution', 'Others', 'Overflow of Storm Water Drains', 'Parking Violations', 'Parks and playgrounds', 'Potholes', 'Public Nuisance', 'Repair of streetlights', 'Sewage and Storm Water Drains - Others', 'Stray Dogs', 'Traffic', 'Trees, Parks and Playgrounds - Others', 'Unauthorized Construction', 'Water Leakage', 'Water Supp

In [18]:
keywords = {'Air Pollution':['dust|smoke|burn'],
 'Autorickshaws and Taxis':['taxi','auto','autorickshaw'],
 'BMTC - Driver or Conductor':['rude','behaviour'],
 'BMTC - Need new Bus Route':['frequency'],
 'BMTC - Others':[],
 'Bad Roads':['road'],
 'Broken Storm Water Drains':['overflow','drainage'],
 'Cattle':['cows','buffaloes','goats','cow'],
 'Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover':[],
 'Desilting - Lakes':['lake'],
 'Diseases':['malaria','dengue','cholera','fever','disease','hospital','epidemic'],
 'Electricity':['power','current','power cut'],
 'Flooding of Roads and Footpaths':['water','flood','floods'],
 'Footpaths':['footpath'],
 'Garbage':['waste','plastic','dirt'],
 'Government Land Encroachment':['occupy','illegal'],
 'Hawkers and Vendors':[],
 'Hoardings':['advertise'],
 'Illegal posters and Hoardings':['banner', 'ads ','advertise'],
 'Lakes - Others':['lake'],
 'Maintenance of Roads and Footpaths - Others':[],
 'Manholes':['manhole','man hole'],
 'Mosquitos':['mosquito','mosquitoe','mosquitoes','dengue','malaria'],
 'Need New Streetlights':['streetlight','light','new streetlight'],
 'Need New Toilets ':['toilet','urinal','urinate'],
 'New Bus Shelters':['shelter'],
 'No Sewage Drains':['drainage'],
 'Noise Pollution':['siren','speakers','speakers','loud'],
 'Others':[],
 'Overflow of Storm Water Drains':['pipes'],
 'Parking Violations':['parked','parker'],
 'Parks and playgrounds':['park','play','playground'],
 'Potholes':['holes','pothole'],
 'Public Nuisance':[],
 'Repair of streetlights':['streetlight','light','broken','damaged'],
 'Sewage and Storm Water Drains - Others':['drainage'],
 'Stray Dogs':['dog'],
 'Traffic':['vehicles'],
 'Trees, Parks and Playgrounds - Others':['tree'],
 'Unauthorized Construction':['encroach','building','built'],
 'Water Leakage':[],
 'Water Supply ':[]}

In [19]:
regexPatterns = {'Air Pollution':['air.*pollution|pollution|dust'],
 'Autorickshaws and Taxis':['autorickshaws|taxis|taxi|auto|autorickshaw'],
 'BMTC - Driver or Conductor':['bmtc.*driver|bmtc.*conductor|bus.*driver|bus.*conductor'],
 'BMTC - Need new Bus Route':['bus.*route'],
 'BMTC - Others':['bmtc'],
 'Bad Roads':['bad.*road|road.*bad'],
 'Broken Storm Water Drains':['(broken|damage).*(drain)'],
 'Cattle':['(cattle|cows|buffaloes|goats)'],
 'Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover':['clearing|blockage|under|ground|drainage|pipelines|replacement|damaged|missing|manhole|cover'],
 'Desilting - Lakes':['lake'],
 'Diseases':['diseases|malaria|dengue|cholera'],
 'Electricity':['electricity|power|current|power.*cut'],
 'Flooding of Roads and Footpaths':['((water|flood|flow).*(roads|footpaths))|((roads|footpaths).*(water|flood|flow))'],
 'Footpaths':['footpath'],
 'Garbage':['garbage|waste|plastic|dirt'],
 'Government Land Encroachment':['(government.*land).*(encroach|occupy|illegal)'],
 'Hawkers and Vendors':['(hawkers|vendors)'],
 'Hoardings':['(hoardings|advertisements)'],
 'Illegal posters and Hoardings':['posters|hoardings|banner|ads|advertise'],
 'Lakes - Others':['lake'],
 'Maintenance of Roads and Footpaths - Others':['(maintenance).*(roads|footpaths)'],
 'Manholes':['(manholes|manhole|man hole)'],
 'Mosquitos':['mosquito|mosquitoe|mosquitoes|dengue|malaria'],
 'Need New Streetlights':['(need|no|new).*(streetlight|light)'],
 'Need New Toilets ':['toilets|toilet|urinal|urinate'],
 'New Bus Shelters':['bus.*shelter|shelter.*bus'],
 'No Sewage Drains':['drain'],
 'Noise Pollution':['noise|noise.*pollution|siren|speakers|speakers|loud'],
 'Others':['others'],
 'Overflow of Storm Water Drains':['overflow.*(drains|pipes)'],
 'Parking Violations':['parking|parked|parker'],
 'Parks and playgrounds':['(parks|playgrounds|park|play|playground)'],
 'Potholes':['(pot hole|holes|pothole)'],
 'Public Nuisance':['(public.*nuisance|nuisance)'],
 'Repair of streetlights':['((light).*(repair|broke|damage))|((repair|broke|damage).*(light))'],
 'Sewage and Storm Water Drains - Others':['(sewage|storm|water|drains|drainage)'],
 'Stray Dogs':['(stray|dogs|dog)'],
 'Traffic':['(traffic|vehicles)'],
 'Trees, Parks and Playgrounds - Others':['(trees|parks|playgrounds|tree)'],
 'Unauthorized Construction':['encroach','building','built'],
 'Water Leakage':['water.*leak|leak.*water'],
 'Water Supply ':['water.*supply|supply.*water']}

In [20]:
extracts_df=pd.read_csv('./p.tsv',sep='\t',usecols=["category_name","Entity","complaint words",],na_filter=False)

extracts_df = extracts_df[extracts_df['category_name'].isin(class_names)]
# extracts_df
def combine(x):
    x = x.tolist()
    x = set(x)
    x = '|'.join(list(x)).lower()
    return x
extracts_df = extracts_df.groupby("category_name").agg({"Entity": combine,"complaint words":combine })
extracts_df.to_csv("extracts.csv")
extracts_df


,Entity,complaint words
category_name,,
Air Pollution,bus|garbage,lot of dust due to speeding buses|wood or garb...
BMTC - Driver or Conductor,gbus,rude behavious
BMTC - Need new Bus Route,bus,increase frequency of buses
BMTC - Others,bus|traffic,need more bmtc buses|traffic jams|increase in ...
Bad Roads,roads|pothole|roads|permit|sewage|footpath,road in pathetic state|road bumpy and muddy|ho...
Broken Storm Water Drains,drain,drainage left open|no drainage lane|storm wate...
Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover,tank|sewage,sewage drain blocked|connect tank to drainage ...
Desilting - Lakes,lake,clean lajke
Diseases,cat|garbage,dead cat at residential place|garbage|garbage ...


In [21]:
import re
class_words = [ re.sub('-','',x).lower().split() + keywords[x] for x in class_names ]
print(class_words,len(class_words))

[['air', 'pollution', 'dust|smoke|burn'], ['autorickshaws', 'and', 'taxis', 'taxi', 'auto', 'autorickshaw'], ['bmtc', 'driver', 'or', 'conductor', 'rude', 'behaviour'], ['bmtc', 'need', 'new', 'bus', 'route', 'frequency'], ['bmtc', 'others'], ['bad', 'roads', 'road'], ['broken', 'storm', 'water', 'drains', 'overflow', 'drainage'], ['cattle', 'cows', 'buffaloes', 'goats', 'cow'], ['clearing', 'of', 'blockage', 'of', 'under', 'ground', 'drainage', 'pipelines', 'and', 'replacement', 'of', 'damaged', 'or', 'missing', 'manhole', 'cover'], ['desilting', 'lakes', 'lake'], ['diseases', 'malaria', 'dengue', 'cholera', 'fever', 'disease', 'hospital', 'epidemic'], ['electricity', 'power', 'current', 'power cut'], ['flooding', 'of', 'roads', 'and', 'footpaths', 'water', 'flood', 'floods'], ['footpaths', 'footpath'], ['garbage', 'waste', 'plastic', 'dirt'], ['government', 'land', 'encroachment', 'occupy', 'illegal'], ['hawkers', 'and', 'vendors'], ['hoardings', 'advertise'], ['illegal', 'posters', 

In [22]:
########### discrete LFs ####
import os
import re

stopwords_pattern = ' of| and| no| others| or| -|,|no '

def ltp(x):
    return '(' + '|'.join(x) + ')'

def create_LF_Based_On_Category_Name(debug=False):
    if os.path.exists("d_Category_Name_LFs.py"):
        os.remove("d_Category_Name_LFs.py")
        
    f = open("d_Category_Name_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])

        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
        pattern= re.sub(" $","",pattern)
        
        words = pattern.split(',')
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Category_Name_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    if(len(set(c['complaint_text']).intersection(words))>0):
            return '''+str(i+1)+'''
    return 0''')
        f.write("\n")
    f.close()
    


    

def create_LF_Based_On_Keywords(debug=False):
    if os.path.exists("d_KeyWord_Based_LFs.py"):
        os.remove("d_KeyWord_Based_LFs.py")
    f = open("d_KeyWord_Based_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_KeyWord_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    if(len(set(c['complaint_text']).intersection(words))>0):
            return '''+str(i+1)+'''
    return 0''')
        f.write("\n")
    f.close()
    
def create_LF_Extracts_Phrases_Regex(debug=False):
    if os.path.exists("d_Regex_Based_Extracts_Phrases_LFs.py"):
        os.remove("d_Regex_Based_Extracts_Phrases_LFs.py")
    f = open("d_Regex_Based_Extracts_Phrases_LFs.py","a+")
    for i in range(len(class_names)):
        if(class_names[i] in extracts_df.index.tolist()):
            functionName = re.sub(r'( )+|-|,','',class_names[i])

            pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
            pattern= re.sub("( )+",",",pattern)

            words = pattern.split(',')
            ##### add keywords #####

            words = words+ keywords[class_names[i]]
            ####
            wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'


            if(debug):
                print(pattern)
            f.write("\n")
            f.write(r'''def LF_Extract_Phrase_Regex_'''+functionName+'''(c):
   
    pattern = \''''+extracts_df.loc[class_names[i]]['complaint words']+'''\'
    if(re.search(pattern,c['complaint_description'],flags=re.I)):
            return '''+str(i+1)+'''
    return 0''')
            f.write("\n")
    f.close()
    
def create_LF_Description_Regex(debug=False):
    if os.path.exists("d_Regex_Based_Description_LFs.py"):
        os.remove("d_Regex_Based_Description_LFs.py")
    f = open("d_Regex_Based_Description_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
       
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Desc_Regex_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    pattern = \''''+''.join(regexPatterns[class_names[i]])+'''\'
    if(re.search(pattern,c['complaint_description'],flags=re.I)):
            return '''+str(i+1)+'''
    return 0''')
        f.write("\n")
    f.close()
    
    
    
def create_LF_Title_Regex(debug=False):
    if os.path.exists("d_Regex_Based_Title_LFs.py"):
        os.remove("d_Regex_Based_Title_LFs.py")
    f = open("d_Regex_Based_Title_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Title_Regex_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    pattern = \''''+ ''.join(regexPatterns[class_names[i]]) +'''\'
    if(re.search(pattern,c['complaint_title'],flags=re.I)):
            return '''+str(i+1)+'''
    return 0''')
        f.write("\n")
    f.close()


In [23]:
LF_Names = []
LF_output_map = dict()
    
create_LF_Title_Regex()
create_LF_Description_Regex()
# create_LF_Based_On_Keywords()
# create_LF_Extracts_Phrases_Regex()
# create_LF_Based_On_Category_Name()
# create_LF_Based_On_Embeddings()
# create_LF_Based_On_TFIDF()
# create_LF_Based_On_Embeddings_Title()
# create_LF_Based_On_Embeddings_Description()



        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Category_Name_'+functionName)
#         LF_output_map['LF_Category_Name_'+functionName]=i
        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Embedding_'+functionName)
#         LF_output_map['LF_Embedding_'+functionName]=i
        

        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_TFIDF_'+functionName)
#         LF_output_map['LF_TFIDF_'+functionName]=i

LF_l=[]

# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_KeyWord_'+functionName)
#         LF_output_map['LF_KeyWord_'+functionName]=i
#         LF_l.append(i)

for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_Title_Regex_'+functionName)
        LF_output_map['LF_Title_Regex_'+functionName]=i
        LF_l.append(i)

for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_Desc_Regex_'+functionName)
        LF_output_map['LF_Desc_Regex_'+functionName]=i
        LF_l.append(i)

# for i in range(len(class_names)):
#     if(class_names[i] in extracts_df.index.tolist()):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Extract_Phrase_Regex_'+functionName)
#         LF_output_map['LF_Extract_Phrase_Regex_'+functionName]=i
#         LF_l.append(i)
        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Title_Embedding_'+functionName)
#         LF_output_map['LF_Title_Embedding_'+functionName]=i
        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Description_Embedding_'+functionName)
#         LF_output_map['LF_Description_Embedding_'+functionName]=i
 

        

print('['+','.join(LF_Names)+']')


[LF_Title_Regex_AirPollution,LF_Title_Regex_AutorickshawsandTaxis,LF_Title_Regex_BMTCDriverorConductor,LF_Title_Regex_BMTCNeednewBusRoute,LF_Title_Regex_BMTCOthers,LF_Title_Regex_BadRoads,LF_Title_Regex_BrokenStormWaterDrains,LF_Title_Regex_Cattle,LF_Title_Regex_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_Title_Regex_DesiltingLakes,LF_Title_Regex_Diseases,LF_Title_Regex_Electricity,LF_Title_Regex_FloodingofRoadsandFootpaths,LF_Title_Regex_Footpaths,LF_Title_Regex_Garbage,LF_Title_Regex_GovernmentLandEncroachment,LF_Title_Regex_HawkersandVendors,LF_Title_Regex_Hoardings,LF_Title_Regex_IllegalpostersandHoardings,LF_Title_Regex_LakesOthers,LF_Title_Regex_MaintenanceofRoadsandFootpathsOthers,LF_Title_Regex_Manholes,LF_Title_Regex_Mosquitos,LF_Title_Regex_NeedNewStreetlights,LF_Title_Regex_NeedNewToilets,LF_Title_Regex_NewBusShelters,LF_Title_Regex_NoSewageDrains,LF_Title_Regex_NoisePollution,LF_Title_Regex_Others,LF_Title_Regex_Overflowof

In [36]:
%load d_KeyWord_Based_LFs.py

In [7]:
%load d_Regex_Based_Title_LFs.py

In [9]:
%load d_Regex_Based_Description_LFs.py

In [42]:
%load d_Regex_Based_Extracts_Phrases_LFs.py

In [24]:
LFs = [LF_Title_Regex_AirPollution,LF_Title_Regex_AutorickshawsandTaxis,LF_Title_Regex_BMTCDriverorConductor,LF_Title_Regex_BMTCNeednewBusRoute,LF_Title_Regex_BMTCOthers,LF_Title_Regex_BadRoads,LF_Title_Regex_BrokenStormWaterDrains,LF_Title_Regex_Cattle,LF_Title_Regex_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_Title_Regex_DesiltingLakes,LF_Title_Regex_Diseases,LF_Title_Regex_Electricity,LF_Title_Regex_FloodingofRoadsandFootpaths,LF_Title_Regex_Footpaths,LF_Title_Regex_Garbage,LF_Title_Regex_GovernmentLandEncroachment,LF_Title_Regex_HawkersandVendors,LF_Title_Regex_Hoardings,LF_Title_Regex_IllegalpostersandHoardings,LF_Title_Regex_LakesOthers,LF_Title_Regex_MaintenanceofRoadsandFootpathsOthers,LF_Title_Regex_Manholes,LF_Title_Regex_Mosquitos,LF_Title_Regex_NeedNewStreetlights,LF_Title_Regex_NeedNewToilets,LF_Title_Regex_NewBusShelters,LF_Title_Regex_NoSewageDrains,LF_Title_Regex_NoisePollution,LF_Title_Regex_Others,LF_Title_Regex_OverflowofStormWaterDrains,LF_Title_Regex_ParkingViolations,LF_Title_Regex_Parksandplaygrounds,LF_Title_Regex_Potholes,LF_Title_Regex_PublicNuisance,LF_Title_Regex_Repairofstreetlights,LF_Title_Regex_SewageandStormWaterDrainsOthers,LF_Title_Regex_StrayDogs,LF_Title_Regex_Traffic,LF_Title_Regex_TreesParksandPlaygroundsOthers,LF_Title_Regex_UnauthorizedConstruction,LF_Title_Regex_WaterLeakage,LF_Title_Regex_WaterSupply,LF_Desc_Regex_AirPollution,LF_Desc_Regex_AutorickshawsandTaxis,LF_Desc_Regex_BMTCDriverorConductor,LF_Desc_Regex_BMTCNeednewBusRoute,LF_Desc_Regex_BMTCOthers,LF_Desc_Regex_BadRoads,LF_Desc_Regex_BrokenStormWaterDrains,LF_Desc_Regex_Cattle,LF_Desc_Regex_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_Desc_Regex_DesiltingLakes,LF_Desc_Regex_Diseases,LF_Desc_Regex_Electricity,LF_Desc_Regex_FloodingofRoadsandFootpaths,LF_Desc_Regex_Footpaths,LF_Desc_Regex_Garbage,LF_Desc_Regex_GovernmentLandEncroachment,LF_Desc_Regex_HawkersandVendors,LF_Desc_Regex_Hoardings,LF_Desc_Regex_IllegalpostersandHoardings,LF_Desc_Regex_LakesOthers,LF_Desc_Regex_MaintenanceofRoadsandFootpathsOthers,LF_Desc_Regex_Manholes,LF_Desc_Regex_Mosquitos,LF_Desc_Regex_NeedNewStreetlights,LF_Desc_Regex_NeedNewToilets,LF_Desc_Regex_NewBusShelters,LF_Desc_Regex_NoSewageDrains,LF_Desc_Regex_NoisePollution,LF_Desc_Regex_Others,LF_Desc_Regex_OverflowofStormWaterDrains,LF_Desc_Regex_ParkingViolations,LF_Desc_Regex_Parksandplaygrounds,LF_Desc_Regex_Potholes,LF_Desc_Regex_PublicNuisance,LF_Desc_Regex_Repairofstreetlights,LF_Desc_Regex_SewageandStormWaterDrainsOthers,LF_Desc_Regex_StrayDogs,LF_Desc_Regex_Traffic,LF_Desc_Regex_TreesParksandPlaygroundsOthers,LF_Desc_Regex_UnauthorizedConstruction,LF_Desc_Regex_WaterLeakage,LF_Desc_Regex_WaterSupply]

In [25]:
pkl.dump(LF_Names,open("LF_Names.p","wb"))
pkl.dump(LF_output_map,open("LF_output_map.p","wb"))
pkl.dump(LF_l,open("LF_l.p","wb"))
print(len(LF_Names))
print(len(LF_output_map))
print(len(LF_l))


84
84
84


In [26]:
# returns: NoOf samples by NoOf LFs list of lists
def get_L_S_Tensor(df,msg): 
    L = []
    print('labelling ',msg,' data')
    for i in range(len(df.index)):
        Li=[]
        for LF in LFs:
#             print(i,LF.__name__)            
            l = LF(df.iloc[i])
            Li.append(l)
#             S.append((s+1)/2)  #to scale scores in [0,1] 
        L.append(Li)
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
        
    return L

import time
import datetime
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))


test_L_S = get_L_S_Tensor(test_df,'discrete test')
pkl.dump(test_L_S,open("test_L_S_regex84.p","wb"))


train_L_S = get_L_S_Tensor(train_df,'discrete train')
pkl.dump(train_L_S,open("train_L_S_regex84.p","wb"))

# gold_L_S = get_L_S_Tensor(gold_df,'discrete gold')
# print(np.array(gold_L_S).shape)
# pkl.dump(gold_L_S,open("gold_discrete.p","wb"))
# print()


print(str(datetime.timedelta(seconds=time.time() - start_time)))


started at: 14-5-2018, 18:1:19
labelling  discrete test  data
500data points labelled in 0.08257432381312052 mins
1000data points labelled in 0.16361392736434938 mins
1500data points labelled in 0.2449163516362508 mins
2000data points labelled in 0.3271337231000265 mins
2500data points labelled in 0.407572074731191 mins
3000data points labelled in 0.4874963998794556 mins
3500data points labelled in 0.5673590540885926 mins
4000data points labelled in 0.6511170705159505 mins
4500data points labelled in 0.7337201515833537 mins
5000data points labelled in 0.8158075094223023 mins
5500data points labelled in 0.8980512579282125 mins
6000data points labelled in 0.9775981227556865 mins
6500data points labelled in 1.0569812099138896 mins
labelling  discrete train  data
500data points labelled in 1.1872913360595703 mins
1000data points labelled in 1.287822381655375 mins
1500data points labelled in 1.388413699467977 mins
2000data points labelled in 1.4905155261357625 mins
2500data points labelled 

In [30]:
import scipy.sparse as sp
import _pickle as pkl
# L_train = pkl.load(open("train_L_S_discrete.p","rb"))
# L_train = sp.csr_matrix(L_train)

# L_gold = pkl.load(open("gold_discrete.p","rb"))
# print(np.array(L_gold).shape)
# L_gold = sp.csr_matrix(L_gold)

L_train = pkl.load(open("train_L_S_regex84.p","rb"))
print(np.array(L_train).shape)
L_train = sp.csr_matrix(L_train)

L_gold = pkl.load(open("test_L_S_regex84.p","rb"))
print(np.array(L_gold).shape)
L_gold = sp.csr_matrix(L_gold)

(37595, 84)
(6667, 84)


In [31]:
import os
import numpy as np

from snorkel import SnorkelSession
session = SnorkelSession()

In [32]:
from snorkel.learning import GenerativeModel
from sklearn.externals import joblib
gen_model = GenerativeModel()


Instructions for updating:
Use the retry module or similar alternatives.


In [50]:
# Note: We pass cardinality explicitly here to be safe
# Can usually be inferred, except we have no labels with value=3

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train, cardinality=42)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)
joblib.dump(clf, 'snorkel_gen_model.pkl') 

print(str(datetime.timedelta(seconds=time.time() - start_time)))

started at: 13-4-2018, 23:2:25
1:32:12.027037


In [ ]:
# rerun
# Note: We pass cardinality explicitly here to be safe
# Can usually be inferred, except we have no labels with value=3

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train, cardinality=42)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)
# joblib.dump(gen_model, 'snorkel_gen_model.pkl') 

print(str(datetime.timedelta(seconds=time.time() - start_time)))

In [79]:
train_marginals = gen_model.marginals(L_train)
train_marginals = np.array(train_marginals)
train_GenLabels = np.argmax(train_marginals,axis=1)
print(train_GenLabels.shape)
train_GenLabels = [class_names[x] for x in train_GenLabels]
train_df["predicted_label"]=train_GenLabels
train_df.to_csv("ICMCwithSnorkelPredictedLabels.csv")

(37595,)


In [50]:
gold_marginals = gen_model.marginals(L_gold)
gold_marginals = np.array(gold_marginals)
gold_GenLabels = np.argmax(gold_marginals,axis=1)

gold_GenLabels = [class_names[x] for x in gold_GenLabels]

gold_df_out=pd.read_csv('./clean-gold-labels.tsv',sep='\t',na_filter=False)
gold_df_out["predicted_label"]=gold_GenLabels
gold_df_out.to_csv("500ExampleswithSnorkelPredictedLabels.csv")

In [68]:
L_test = pkl.load(open("test_L_S_discrete.p","rb"))
L_test = sp.csr_matrix(L_test)

In [72]:
import numpy as np
dev_marginals = gen_model.marginals(L_test)
dev_marginals = np.array(dev_marginals)
print(dev_marginals.shape)

GenLabels = np.argmax(dev_marginals,axis=1)
print(GenLabels.shape)


(6667, 42)
(6667,)


In [59]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(true_labels,pl):
    print("acc",accuracy_score(true_labels,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
#     draw2DArray(confusion_matrix(true_labels,pl))
    return report2dict(classification_report(true_labels, pl, target_names=class_names))
    

In [65]:
#load true test labels
import pandas as pd

test_df=pd.read_csv('./complaints_test_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

colsize = len(test_df['category_name'])

test_df['category_name'] = test_df["category_name"].astype('category')
test_df['true_label'] = test_df['category_name'].cat.codes

true_labels = test_df['true_label'].tolist()
test_df

,category_name,complaint_title,complaint_description,true_label
0,Maintenance of Roads and Footpaths - Others,Mud road yet to be tarred,We had a nice tar road in Kannamangala connect...,20
1,Public Nuisance,Policemen creating discomfort,Most of the time some 2 policemen from Bangalo...,33
2,Hoardings,Illegal hoardings on bellary road,Illegal hoardings on bellary road on servies r...,17
3,Repair of streetlights,Street Light Unavailable for several Years,"In our layout, Margosa Avenue opposite next to...",34
4,Traffic,Traffic issues near KG Halli entrance,Traffic issues near KG Halli entrance,37
5,Bad Roads,"No Tar road near Vaswani Centropolis, Shanti N...",There is no Tar road beside Vaswani Centropolis.,5
6,Garbage,Daily garbage left on road,Daily garbage collected on road and uncleared,14
7,Potholes,"Fill pot holes on CMH Road,Indiranagar,Hoysala...",There are lot of pot holes on Chinmayya missio...,32
8,Sewage and Storm Water Drains - Others,Irresponsible &amp; Apathetic : &lt; 100\' fee...,Drain construction work in front of Ittina Sou...,35
9,Garbage,Dumping of garbage,Garbage is dumped near our compound which has ...,14


In [60]:
#snorkel
Results = predictAndPrint(true_labels,GenLabels)
Results

acc 0.568021598920054


,f1-score,precision,recall,support
BMTC - Driver or Conductor,0.42,0.67,0.31,26.0
Cattle,0.50,0.57,0.44,9.0
Diseases,0.10,0.25,0.06,49.0
Government Land Encroachment,0.11,0.09,0.12,8.0
Lakes - Others,0.24,0.14,0.73,11.0
Manholes,0.27,0.19,0.46,13.0
New Bus Shelters,0.30,0.53,0.21,121.0
No Sewage Drains,0.26,0.19,0.41,100.0
Others,0.00,0.00,0.00,131.0
Overflow of Storm Water Drains,0.00,0.00,0.00,41.0


In [75]:
#majority
import math
L_test = pkl.load(open("test_L_S_discrete.p","rb"))
L_test = pd.DataFrame(L_test)
L_test = L_test.replace(0, np.NaN)
predicted_labels=L_test.mode(axis=1)[0].tolist()
# print(predicted_labels)
predicted_labels = [ int(x)-1 if not math.isnan(x) else -1 for x in predicted_labels ]
MajLabels = predicted_labels
Results = predictAndPrint(true_labels,predicted_labels)
Results

acc 0.5696715164241788


/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 43, does not match size of target_names, 42
  .format(len(labels), len(target_names))
/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,f1-score,precision,recall,support
BMTC - Driver or Conductor,0.41,0.27,0.83,18.0
Cattle,0.07,0.12,0.05,20.0
Diseases,0.20,0.12,0.70,10.0
Government Land Encroachment,0.88,0.88,0.88,1319.0
Lakes - Others,0.21,0.13,0.64,14.0
Manholes,0.00,0.00,0.00,27.0
New Bus Shelters,0.60,0.59,0.62,21.0
No Sewage Drains,0.35,0.93,0.21,121.0
Others,0.54,0.57,0.52,31.0
Overflow of Storm Water Drains,0.00,0.00,0.00,131.0
